In [1]:
!pip install chromadb
!pip install pypdf
!pip install pyspellchecker

In [2]:
from pathlib import Path
import os
import shutil

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

from autocorrect import Speller


In [3]:
SENTENCE_MODEL_NAME = "ai-forever/sbert_large_nlu_ru"
# LLM_MODEL_NAME = "ai-forever/FRED-T5-1.7B"
# LLM_MODEL_NAME = "ai-forever/FRED-T5-large"
# LLM_MODEL_FOLDER = "TheBloke/Llama-2-7B-GGUF"
LLM_MODEL_NAME = "llama-2-7b.Q4_K_M.gguf"

In [4]:
WORKDIR_PATH = Path.home()
DOCS_PATH = WORKDIR_PATH / "data"
MODEL_PATH = WORKDIR_PATH / "models"
CHROMA_PATH = WORKDIR_PATH / "chromadb"

COLLECTION_NAME = "articles"

In [5]:
# TODO: стоит разобрать как делать кастомные токены и стоит ли
JOIN_TOKEN = "\n\n---------------\n\n"
METADATA_TOKEN = "\n-----\n"

In [6]:
chunk_size = 1000
chunk_overlap = 300
# set_cache_dir(MODEL_PATH)

# Разворот моделей

## sentencetransformer

In [7]:
embedding_function = SentenceTransformerEmbeddings(model_name=SENTENCE_MODEL_NAME)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/jovyan/.cache/torch/sentence_transformers/ai-forever_sbert_large_nlu_ru. Creating a new one with MEAN pooling.
/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Удаление старой vectordb

In [8]:
# shutil.rmtree(CHROMA_PATH)
# os.makedirs(CHROMA_PATH)

# Инициализация векторной db

In [9]:
def load_documents(path = DOCS_PATH):
    # spell = Speller("ru")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # load the document and split it into chunks
    # pdf_paths_list = [str(doc) for doc in Path(path).glob("*.pdf")]
    for pdf in pdf_paths_list:
        print(f"current docs: {pdf}")
        # loader = PyPDFLoader(pdf)
        with open("../../state_of_the_union.txt") as f:
            state_of_the_union = f.read()
        documents = loader.load()
        # split it into chunks
        docs = text_splitter.split_documents(documents)
        # clean with spellchecker
        # for doc in docs:
        #     doc.page_content = spell(doc.page_content)
        
        # load it into Chroma
        Chroma.from_documents(
            docs,
            embedding_function,
            collection_name=COLLECTION_NAME,
            persist_directory=str(CHROMA_PATH)
        )

In [10]:
load_documents()

current docs: /home/jovyan/data/6 О подходах к изучению субъективного благополучия личности в зарубежных исследованиях .pdf
current docs: /home/jovyan/data/5. Мат.модель математ.логики - статья.pdf
current docs: /home/jovyan/data/3 О роли сознания в субъективном благополучии личности.pdf
current docs: /home/jovyan/data/1 Субъективное благополучие студентов с разным уровнем социального капитала.pdf
current docs: /home/jovyan/data/4 Субъективное благополучие личности в контексте жизненных стратегий.pdf
current docs: /home/jovyan/data/1.Петренко В.Ф. - Основы психосемантики.pdf
current docs: /home/jovyan/data/10.ЭВРИСТ. ИИ. ЗАРУБЕЖ.ПуБЛИКАЦИЯ.pdf
current docs: /home/jovyan/data/6. Диалектика и мат.логика - статья.pdf
current docs: /home/jovyan/data/5 Субъективное благополучие как проблема социальной психологии личности.pdf
current docs: /home/jovyan/data/7 Факторы, определяющие субъективное благополучие личности.pdf
current docs: /home/jovyan/data/11 Субъективное благополучие и эмоциональ

# Тесты

In [13]:
chroma_db = Chroma(
        persist_directory=str(CHROMA_PATH),
        embedding_function=embedding_function,
        collection_name=COLLECTION_NAME,
    )

similiartiy_responce = chroma_db.similarity_search("благополучие студента", k=5)
beatify2print = JOIN_TOKEN.join(list(map(lambda x: x.page_content + METADATA_TOKEN + str(x.metadata), similiartiy_responce)))
# .join("\n\n")
# similiartiy_pg_cnt
print(beatify2print)

Психологическое благополучие личности имеет достаточно сложную 
структуру. Кроме того, в данном пон ятии представлены обобщенно успешность 
поведения и деятельности, удовлетворенность общением, межличностными 
связями.  
Таким образом, психологическое благополучие личности — это 
субъективная характеристика, применимая лишь для конкретного человека. 
Психологи ческое благополучие личности, наряду с физическим, необходимо 
рассматривать как одно из самых важных и фундаментальных, поскольку от 
того, как человек чувствует себя в этом мире, нашел ли он своѐ призвание и 
цель в жизни, зависит его дальнейшая жизнедеятел ьность.  
 
Список литературы : 
1. Шпионов  P.M. Психология субъективного благополучия: (к разработке 
интегративной концепции) [Текст]  / P.M. Шпионов // Мир психологии. — 
2012. — № 2. — с. 143—148.
-----
{'page': 5, 'source': '/home/jovyan/data/4 Субъективное благополучие личности в контексте жизненных стратегий.pdf'}

---------------

ДК 15.81.57
М. В. Чумаков
Кургански